In [71]:
from db_utils import *

In [118]:
query = """
SELECT
sum(pageviews) as n,
timestamp, 
country_name, 
project, 
access_method
FROM staging.pentahoviews05
WHERE is_spider = 0
AND is_automata = 0
AND project RLIKE 'wikipedia'
group by timestamp, country_name, project, access_method
"""

d =  query_analytics_store(query, {})

In [119]:
str_dimensions = ['project', 'access_method', 'country_name', 'timestamp']
d[str_dimensions] = d[str_dimensions].applymap(lambda x: x.decode("utf-8"))

In [120]:
d.index = d['timestamp']

In [121]:
group_dimensions = ['project', 'access_method', 'country_name',]
groups = d.groupby(group_dimensions)

In [122]:
cube = {}
for group in groups:
    colname = '/'.join(group[0])
    data = pd.Series(group[1]['n'])
    cube[colname] = data

In [152]:
df = pd.DataFrame(cube)

In [165]:
totals = df.sum(axis=0)
large_wikis = df.sum(axis=0) > 250000
dr = df[list(totals[large_wikis].index)]

In [168]:
dr['YearMonth'] = dr.index

/Users/ellerywulczyn/miniconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [173]:
dr.to_csv('./app/data/cube5.csv', index = False)